In [25]:
using JSON,Random
using OrderedCollections

# Specify the file name and location
filename = "medium_1.json"
# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = false  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]
two_tone_vehicle = []
i = 0
for vehicle in vehicles
    i+=1
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: ")
    else
        # Otherwise, print the actual vehicle type
        if vehicle["type"] == "two-tone"
            push!(two_tone_vehicle,i)
        end
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end

println(two_tone_vehicle)


Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 0), Dict{String, Any}("name" => "paint", "resequencing_lag" => 10), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 15)]
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 5000, "id" => 1, "window_size" => 3, "vehicles" => Any[4, 5, 8, 9, 11, 16, 18, 23, 26, 29, 30, 33, 34, 36, 37, 38, 41, 42, 45, 46, 49, 50, 53, 55, 64, 70, 76, 77, 79, 84, 85, 86, 89, 90, 91, 93, 94, 96, 97, 100, 103, 104, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 122, 123, 125, 127, 128, 134, 135, 138, 141, 144, 146, 148, 149, 150, 152, 154, 155, 157, 158, 159, 165, 166, 170, 171, 178, 179, 180, 181, 184, 188, 190, 191, 192, 195, 196, 197, 199, 201, 203, 204, 205, 211, 213, 215, 216, 217, 218, 219, 220, 223, 225, 226, 227, 230, 231, 233, 235, 237, 239, 240, 241, 243, 250, 253, 255, 256, 257, 258, 259, 265, 266, 268, 269, 272, 274, 276, 277, 279, 280, 281, 283, 285, 286, 291, 293, 294, 297], "type" => "rolling_window",

In [26]:
cs = parameters["resequencing_cost"]
number_of_shops = length(shops)    
δ = parameters["two_tone_delta"]
println(δ)
i=0 
for shop in shops
    i+=1
    
    println("Shop: ",i,"  ", shop)
end
println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

115
Shop: 1  Dict{String, Any}("name" => "body", "resequencing_lag" => 0)
Shop: 2  Dict{String, Any}("name" => "paint", "resequencing_lag" => 10)
Shop: 3  Dict{String, Any}("name" => "assembly", "resequencing_lag" => 15)
Number of shops: 3
number_of_vehicles, 297


In [27]:
function γ(k, c_bb, mb, Mb)
    return c_bb * max(0, mb - k, k - Mb)^2
end

γ (generic function with 1 method)

In [28]:

# Initialize an empty dictionary to store the mapping
shop_map = OrderedDict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
lag_map = OrderedDict{String, Integer}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
# Populate the shop_map with shop names as keys and empty arrays as values
i = 0
for shop in shops
    i+=1
    shop_map[shop["name"]] = []
    lag_map[shop["name"]] = shop["resequencing_lag"]
    println(shop["name"],"i :",i)
end
n = number_of_vehicles
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map

for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

(lag_map)


bodyi :1
painti :2
assemblyi :3
n: 297


OrderedDict{String, Integer} with 3 entries:
  "body"     => 0
  "paint"    => 10
  "assembly" => 15

In [30]:
# Function to perturb a solution (for example, swapping two elements)
function perturb_solution_big(perm)
    new_perm = copy(perm)
    idx1, idx2 = rand(1:length(perm), 2)
    # Swap two random elements
    new_perm[idx1], new_perm[idx2] = new_perm[idx2], new_perm[idx1]
    return new_perm
end

function perturb_solution(perm)
    new_perm = copy(perm)
    # Randomly select an index for the swap, ensuring it's not the last index
    idx = rand(1:length(perm) - 1)
    # Swap the element at idx with the element at idx + 1
    new_perm[idx], new_perm[idx + 1] = new_perm[idx + 1], new_perm[idx]
    return new_perm
end



perturb_solution (generic function with 1 method)

In [31]:

using Random

function perturb_perms(perms)
    # Create a copy of the perms list to avoid modifying the original list
    perm_copy = copy(perms)

    # Select a random index
    i = rand(1:length(perms))

    # Extract the permutation at index i
    perm = perms[i]

    # Apply perturbation to the selected permutation
    perturbed_perm = perturb_solution(perm)

    # Update the copied list with the perturbed permutation
    perm_copy[i] = perturbed_perm

    return perm_copy
end
    
function shuffle_perms(perms)
    # Create a copy of the perms list to avoid modifying the original list
    perm_copy = copy(perms)

    # Select a random index
    i = rand(1:length(perms))

    # Extract the permutation at index i
    perm = (perms[i])

    # Apply perturbation to the selected permutation
    perturbed_perm = shuffle(perm)

    # Update the copied list with the perturbed permutation
    perm_copy[i] = perturbed_perm

    return perm_copy
end



shuffle_perms (generic function with 1 method)

In [32]:
# Function to generate targeted neighbors

function generate_targeted_neighbors(perm, Vr, Ul,Vb)
    # Start with a random perturbation as a fallback

    perturbed_perm = random_perturbation(perm)
    n = length(perturbed_perm)
    l = length(Vr)
    k = 0
    n = length(perturbed_perm)

    if l > 0
        for i in 1:n 
            if perturbed_perm[i] in Vr
                k += 1
            else
                k = 0
            end

            if k == l && i < n
                # Find the next element not in Vr
                for j in 1:n
                    if perm[j] ∉ Vr
                        # Swap the i-th element with the j-th element
                        perm[i], perm[j] = perm[j], perm[i]
                        break
                    end
                end
                k = 0  # Reset k after the swap
            end
        end
    end
    # Handle perturbation based on Ul
    for i in 1:(n-1)
        # Check if the consecutive elements are not in the same sublist in Ul
        if !in_same_lot(perturbed_perm[i], perturbed_perm[i + 1], Ul)
            # Replace one of the elements with a random element from perm
            for j in 1:length(perm)
                if j != i && j != i + 1 && in_same_lot(perturbed_perm[i], perturbed_perm[j], Ul)
                    # Replace element at index i with the one at index j
                    perturbed_perm[i] = perturbed_perm[j]
                    break  # Exit loop once a change has been made
                end
            end
        end
    end
    # Batch size constraints (Vb, mb, Mb, cb)
    if !isempty(Vb) 
        for i in 1:length(Vb)
            vb = Set(Vb[i])

            
            for t in 1:n-1
                b = true
                if t > 1
                    b = !(perturbed_perm[t-1] in vb) && b
                end
                b  = perturbed_perm[t] in vb && b
                if !b 
                    continue
                end
                for t0 in t:n
                    for k in t:t0
                        b = (perturbed_perm[k] in vb) && b
                    end
                    if !b 
                        continue
                    end
                    if t0 <= n-1
                        b = !(perturbed_perm[t0+1] in vb) && b
                    end
                    if b
                        s = (t0-t) ÷ 2
                        if (t>1) 
                            perturbed_perm[t-1],perturbed_perm[t+s] = perturbed_perm[t+s],perturbed_perm[t-1]
                        end
                    end
                end
            end
        end
    end

    return perturbed_perm
end

# Helper function to check if two elements are in the same sublist in Ul
function in_same_lot(elem1, elem2, Ul)
    for lot in Ul
        if elem1 in lot && elem2 in lot
            return true
        end
    end
    return false
end


# Helper function to apply a random perturbation
function random_perturbation(perm)
    # Randomly choose two indices to swap
    copy_perm = copy(perm)
    idx1, idx2 = rand(1:length(perm)), rand(1:length(perm))
    copy_perm[idx1], copy_perm[idx2] = copy_perm[idx2], copy_perm[idx1]
    return copy_perm
end


random_perturbation (generic function with 1 method)

In [34]:
# Function to compute the composition of two permutations
function compose_permutations(sigma, tau)
    n = length(sigma)
    composed_permutation = zeros(Int, n)
    
    for i in 1:n
        composed_permutation[i] = sigma[tau[i]]
    end
    
    return composed_permutation
end

# Function to compute the inverse of a permutation
function inverse_permutation(perm::Vector{Int})
    n = length(perm)
    inv_perm = zeros(Int, n)  # Initialize the inverse permutation with zeros
    
    # The inverse permutation is obtained by setting inv_perm[perm[i]] = i
    for i in 1:n
        inv_perm[perm[i]] = i
    end
    
    return inv_perm
end

# Delta function as given
function delta(t, n, δ)
    return min(δ, n - t)
end

# Cycle function to perform a cycle in the permutation
function cycle(t, δ, n)
    perm = [i for i in 1:n]  # Start with the identity permutation
    k = delta(t, n, δ)  # Calculate the cycle length
    if (k==0) 
        return perm
    end
    perm[t] = t + k  # Set the first step of the cycle
    while k > 1
        perm[t + k] = t + k - 1  # Set each subsequent step
        k -= 1
    end
    perm[t + 1] = t  # Complete the cycle
    return perm
end
function out_permutation(perm,two_tone_vehicle,δ)
    n = length(perm)
    if length(two_tone_vehicle) == 0
        return perm
    end

    copy_perm = copy(perm)
    inverse_perm = inverse_permutation(copy_perm)
    for k in 1:length(two_tone_vehicle)
        u = inverse_perm[two_tone_vehicle[k]]
        tu = cycle(u,δ,n)
        inverse_perm = compose_permutations(tu,inverse_perm)
    end
    # Compute the inverse permutation
    inverse = inverse_permutation(inverse_perm)
    return inverse

# t_u evolves after every permutation so keep that in mind 
end



out_permutation (generic function with 1 method)

In [35]:
function calculate_resequencing_cost(permutation_0, Δs, cs)
    # Initialize the resequencing cost
    total_cost = 0
    permutations = copy(permutation_0)
    # Number of shops (S)
    S = length(permutations)
    
    # Iterate over each consecutive pair of shops
    for s in 1:(S-1)
        # Get the current shop and the next shop
        perm_s = permutations[s]
        perm_s_next = permutations[s+1]
        if (s==2)
            
            perm = copy(perm_s)

            perm_s = out_permutation(perm,two_tone_vehicle,δ)
        end
        # Iterate through each vehicle
        for v in 1:length(perm_s)
            # Get the position of vehicle v in the current shop (s) and the next shop (s+1)
            t_s = findfirst(isequal(v), perm_s)  # Position of v in shop s
            t_s_next = findfirst(isequal(v), perm_s_next)  # Position of v in shop s+1
            
            # Handle cases where the position is not found (i.e., t_s or t_s_next is nothing)

            
            # Access Δs based on the shop index `s`
            Δs_value = Δs[s]  # Δs indexed by s

                # Calculate penalty if the constraint is violated
            penalty = cs * max(0, t_s - Δs_value - t_s_next)
            total_cost += penalty
        end
    end
    
    return total_cost
end


calculate_resequencing_cost (generic function with 1 method)

In [37]:
function find_list_with_element(x, L)
    return findfirst(sublist -> x in sublist, L)
end


function calculate_cost_origin(perm, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb)
    total_cost = 0
    # Lot change constraints (Cl and Ul)
    if length(Ul)>0
        l = length(Ul)
        
        for i in 1:l
            cl = Cl[i]
            ul = (Ul[i])
            for k in 1:length(perm)-1
                t1 = find_list_with_element(perm[k],ul)
                t2 = find_list_with_element(perm[k+1],ul)
                if (t1!=t2) 
                    total_cost+= cl
                end
            end
        end
    end
    # Rolling window constraints (Vr, Mr, wr, cr)
    if !isempty(Vr) && !isempty(Mr) && !isempty(wr) && !isempty(cr)
        for i in 1:length(Vr)
            vr = Set(Vr[i])
            Mr0 = Mr[i]
            wr0 = wr[i]
            cr0 = cr[i]
            for t in 1:max(n-wr0+1)
                s = 0
                for t0 in t:t+wr0-1
                    s+= perm[t0] in vr
                end
                total_cost += cr0*max(0,(s-Mr0))*max(0,(s-Mr0))
            end
        end
    end

    # Batch size constraints (Vb, mb, Mb, cb)
    if !isempty(Vb) && !isempty(mb) && !isempty(Mb) && !isempty(cb)
        for i in 1:length(Vb)
            vb = Set(Vb[i])
            mb0 = mb[i]
            Mb0 = Mb[i]
            cb0 = cb[i]
            
            for t in 1:n-1
                b = true
                if t > 1
                    b = !(perm[t-1] in vb) && b
                end
                b  = perm[t] in vb && b
                if !b 
                    continue
                end
                for t0 in t:n
                    for k in t:t0
                        b = (perm[k] in vb) && b
                    end
                    if !b 
                        continue
                    end
                    if t0 <= n-1
                        b = !(perm[t0+1] in vb) && b
                    end
                    if b
                        total_cost += cb0 * γ(t0 - t + 1, cb0, mb0, Mb0)
                    end
                end
            end
        end
    end
    
    return total_cost
end

calculate_cost_origin (generic function with 1 method)

In [38]:
function calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs)
    cost = calculate_resequencing_cost(perms, Δs, cs)
    for i  in 1:length(perms) 
        cost+= calculate_cost_origin(perms[i], Cl[i], Ul[i], Vr[i], Mr[i], wr[i], cr[i], mb[i], Mb[i], cb[i], Vb[i])
    end
    return cost
end


calculate_cost (generic function with 1 method)

In [39]:
function simulated_annealing(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs; T0, alpha, max_iter)
    # Initialize variables
    current_cost = calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)  # Calculate cost of the initial solution
    best_perms = copy(perms)  # Best solution found
    best_cost = current_cost  # Best cost found
    T = T0  # Initial temperature
    counter = 0
    
    for iter in 1:max_iter
        if (T<1e-25) 
            break
        end
        neighbor_costs = []
        neighbor_perms = []
        # Generate a set of neighboring solutions (perturbations)
        for i in 1:length(perms)
            perm_copy = copy(perms)
            perm = perms[i]
            perturbed_perm = generate_targeted_neighbors(perm, Vr[i], Ul[i],Vb[i])
            perm_copy[i] = perturbed_perm
            perturbed_cost = calculate_cost(perm_copy, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)
            push!(neighbor_perms,perm_copy)
            push!(neighbor_costs,perturbed_cost)
        end
        # Find the best neighboring solution
        min_cost_idx = argmin(neighbor_costs)
        best_neighbor = neighbor_perms[min_cost_idx]
        best_neighbor_cost = neighbor_costs[min_cost_idx]

        # If the best neighboring solution is better, update the current solution
        if best_neighbor_cost < current_cost
            counter =0 
            perms = best_neighbor
            current_cost = best_neighbor_cost

            # If the new solution is the best, update the best solution
            if best_neighbor_cost < best_cost
                best_perms = copy(best_neighbor)
                best_cost = best_neighbor_cost
                
            end
        else
            # If the new solution is worse, accept it with a certain probability based on temperature
            counter += 1
            if counter<2000
                acceptance_probability = exp(-(best_neighbor_cost - current_cost) / T)
                if rand() < acceptance_probability
                    perms = copy(best_neighbor)
                    current_cost = best_neighbor_cost
                end
            else
                # Larger perturbation for worse solutions
                perms = perturb_perms(perms)
                current_cost = calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)
                
            end
        end

        # Reduce the temperature (cooling schedule)
        T *= alpha

        # Optionally print progress
        if iter % 10== 0
            println("Iteration: $iter, Best Cost: $best_cost, Temperature: $T")
        end
    end

    return best_perms, best_cost
end


simulated_annealing (generic function with 1 method)

In [40]:
# Initialize result and Δs lists
result = []
Δs = []

# Initialize global constraint lists for all shops
Cl = []          # Cost list (Ints)
Ul = []  # Partitions (Array of arrays of Ints)
Vr = []  # Vehicles (Array of arrays of Ints)
Mr = []          # Max vehicles (Ints)
wr = []          # Window size (Ints)
cr = []          # Cost (Ints)
mb = []          # Min vehicles (Ints)
Mb = []          # Max vehicles (Ints)
cb = []          # Cost (Ints)
Vb = []  # Vehicles (Array of Ints)

# Loop through each shop and its constraints
for (shop, constraints) in shop_map
    println("shop: ", shop)
    
    # Initialize shop-specific constraint lists
    Cl0 = Int[]          # Cost list (Ints)
    Ul0 = Vector{Vector{Int}}[]  # Partitions (Array of arrays of Ints)
    Vr0 = Vector{Int}[]  # Vehicles (Array of arrays of Ints)
    Mr0 = Int[]          # Max vehicles (Ints)
    wr0 = Int[]          # Window size (Ints)
    cr0 = Int[]          # Cost (Ints)
    mb0 = Int[]          # Min vehicles (Ints)
    Mb0 = Int[]          # Max vehicles (Ints)
    cb0 = Int[]          # Cost (Ints)
    Vb0 = []  # Vehicles (Array of Ints)
    
    # Add the lag for the current shop
    push!(Δs, lag_map[shop])
    
    # Loop through constraints and separate them by type
    for constraint in constraints
        if constraint["type"] == "lot_change"
            cl = constraint["cost"]
            Ul_0 = constraint["partition"]
            push!(Ul0, Ul_0)  # Ensure Ul_0 is an array of Int
            push!(Cl0, cl)
        end

        if constraint["type"] == "rolling_window"
            Vr_0 = constraint["vehicles"]
            Mr_0 = constraint["max_vehicles"]
            wr_0 = constraint["window_size"]
            cr_0 = constraint["cost"]
            push!(Vr0, Vr_0)  # Ensure Vr_0 is an array of Int
            push!(Mr0, Mr_0)
            push!(wr0, wr_0)
            push!(cr0, cr_0)
        end

        if constraint["type"] == "batch_size"
            mb_0 = constraint["min_vehicles"]
            Mb_0 = constraint["max_vehicles"]
            Vb_0 = constraint["vehicles"]
            cb_0 = constraint["cost"]
            push!(mb0, mb_0)
            push!(Mb0, Mb_0)
            push!(Vb0, Vb_0)  # Ensure Vb_0 is an array of Int
            push!(cb0, cb_0)
        end
    end

    # Push all constraint lists (even if they are empty) to the global lists
    push!(Cl, Cl0)
    push!(Ul, Ul0)
    push!(Vr, Vr0)
    push!(Mr, Mr0)
    push!(wr, wr0)
    push!(cr, cr0)
    push!(mb, mb0)
    push!(Mb, Mb0)
    push!(cb, cb0)
    push!(Vb, Vb0)
end


shop: body
shop: paint
shop: assembly


In [72]:
perms = a
current_cost = calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)


8183460

In [ ]:

a,b = simulated_annealing(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs; T0=1000, alpha=0.99, max_iter=100000)

println("total_cost, ",b)
current_cost = calculate_cost(a, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)
println("current cost is, ",current_cost)


Iteration: 10, Best Cost: 8183460, Temperature: 904.3820750088043
Iteration: 20, Best Cost: 8183460, Temperature: 817.9069375972307
Iteration: 30, Best Cost: 8183460, Temperature: 739.7003733882801
Iteration: 40, Best Cost: 8183460, Temperature: 668.9717585696801
Iteration: 50, Best Cost: 8183460, Temperature: 605.006067137536
Iteration: 60, Best Cost: 8183460, Temperature: 547.1566423907609
Iteration: 70, Best Cost: 8183460, Temperature: 494.83865960020677
Iteration: 80, Best Cost: 8183460, Temperature: 447.5232137638104


In [43]:
#current_cost
#large_1 7208100,6830300,6712100,6528300,4617800
#large_2 1377000,1037000,
#medium_1 49740840
#medium_2
#small_1 533400
#small_2 508200


#current_cost = calculate_cost(a, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)



In [ ]:

# Create a mapping with "medium_1 cost" as the key and current_cost as the value,
# and "example_filename medium_1 instance" as the key and `a` as the value.
mapped_content = OrderedDict(
    "$filename cost" => current_cost,
    "$(filename)  instance" => a
)

# Convert the mapping to JSON format
json_data = JSON.json(mapped_content)

# Read the current content of the file (if it exists)
existing_data = ""
if isfile("output.json")
    existing_data = read("output.json", String)
end

# Combine the new info and the existing data (if any)
new_content = json_data * "\n" * existing_data

# Open the file in write mode and overwrite it with the combined content
open("output.json", "w") do file
    write(file, new_content)
end


In [59]:
a

3-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
 [1, 2, 3, 4, 5, 6, 8, 7, 9, 10  …  290, 289, 280, 291, 292, 293, 294, 295, 296, 297]
 [1, 21, 14, 19, 17, 6, 27, 16, 11, 3  …  220, 292, 277, 230, 262, 271, 256, 253, 297, 269]

In [46]:
current_cost

15847640

In [71]:
perms = a
cost = 0
for i  in 1:length(perms)
    cost = 0 
    cost+= calculate_cost_origin(perms[i], Cl[i], Ul[i], Vr[i], Mr[i], wr[i], cr[i], mb[i], Mb[i], cb[i], Vb[i])
    println(cost)
end


39660
2553800
2410000


In [67]:
cost = calculate_resequencing_cost(a, Δs, cs)


3180000